In [176]:
import numpy as np
import jax
import jax.numpy as jnp
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, io as bkio
from bokeh.colors import RGB as bkRGB
import matplotlib as mpl, matplotlib.pyplot as plt
from bokeh import io as bkio, palettes as bkpalettes
from bokeh.colors import RGB as bkRGB

# set up jax
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)

# set up bokeh
bkio.reset_output()
bkplot.output_notebook()

Loading BokehJS ...

In [177]:
from common.selfact_jointexp import bifcurves_intersection

In [178]:
def F1_from_F2(F2, Fb, eta, Qdash, Q1, Q2, M, n, K, I):
    p1 = (K/I)*((1-F2)/(F2-Fb))**(1/eta)
    F1 = p1/(M/n-p1)*(F2*Q2/Q1+(1+Qdash)/Q1)
    return F1

def F2_from_F1(F1, Fb, eta, Qdash, Q1, Q2, M, n, K, I):
    p2 = (K/I)*((1-F1)/(F1-Fb))**(1/eta)
    F2 = p2/(M/n-p2)*(F1*Q1/Q2+(1+Qdash)/Q2)
    return F2

In [179]:
# set parameters
M=1.19e9
n=300
K=250/0.41
I1=0.0025
I2=0.1
eta=2
F0=0.09
Q1=0.4
Q2=0.01
Qdash=0.0

F_range = np.linspace(0.1, 1, 200)

F1s=F1_from_F2(F_range, F0, eta, Qdash, Q1, Q2, M, n, K, I1)
F2s=F2_from_F1(F_range, F0, eta, Qdash, Q1, Q2, M, n, K, I2)

In [180]:
# find intersections for different RC values
Qdash_range = np.linspace(Qdash*0.0, 1, 10)
intersections=[]
F1s=[]
F2s=[]
for i in range(0,len(Qdash_range)):
    Qdash = Qdash_range[i]
    F1s.append(F1_from_F2(F_range, F0, eta, Qdash, Q1, Q2, M, n, K, I1))
    F2s.append(F2_from_F1(F_range, F0, eta, Qdash, Q1, Q2, M, n, K, I2))
    intersections.append(bifcurves_intersection(F_range, F2s[i], F1s[i], F_range))

In [181]:
# plot F curves
curve_no=-1
F_fig = bkplot.figure(
    frame_width=200,frame_height=200,x_range=(0, 1),y_range=(0, 1),x_axis_label="F1",y_axis_label="F2",tools="box_zoom,pan,hover,reset,save"
)
F_fig.output_backend = 'svg'

# plot the F1(F2) curve
F_fig.line(x=F1s[curve_no], y=F_range,
                    line_width=1.5, color='blue',
                    legend_label='F1(F2)')
# plot the F2(F1) curve
F_fig.line(x=F_range, y=F2s[curve_no],
                    line_width=1.5, color='red',
                    legend_label='F2(F1))')

# legend formatting
F_fig.legend.location = "top_right"
F_fig.legend.label_text_font_size = "6pt"
F_fig.legend.padding=0
F_fig.legend.margin=2
F_fig.legend.spacing=0
F_fig.legend.glyph_width=5
F_fig.legend.click_policy = 'hide'

# plot staedy states
ss_fig= bkplot.figure(
    frame_width=200,frame_height=200,x_range=(0, 1),y_range=(0, 1),x_axis_label="F1",y_axis_label="F2",tools="box_zoom,pan,hover,reset,save"
)
ss_fig.output_backend = 'svg'
for i in range(0,len(Qdash_range)):
    ss_fig.scatter(x=intersections[i][0], y=intersections[i][1],
                        line_width=1.5, color=bkpalettes.Plasma11[i])

# plot steady-state resource demands
q_fig= bkplot.figure(
    frame_width=200,frame_height=200,x_range=(0, 1),y_range=(0, 1),x_axis_label="Q'",y_axis_label="Q",tools="box_zoom,pan,hover,reset,save"
)
q_fig.output_backend = 'svg'
for i in range(0,len(Qdash_range)):
    Q=intersections[i][0]*Q1+intersections[i][1]*Q2
    q_fig.scatter(x=Qdash_range[i], y=Q,
                        line_width=1.5, color=bkpalettes.Plasma11[i])

bkplot.show(bklayouts.grid([[F_fig, ss_fig, q_fig]]))